# Mixture of AB Diblock Copolymers and A Homopolymers in Potential Fields

This tutorial demonstrates how to compute partition functions and concentrations for **polymer mixtures** containing different chain architectures.

**What you'll learn:**
- Setting up multi-component polymer systems
- Computing partition functions for each polymer species
- Computing species-specific and total concentrations

**Prerequisites:** Complete `00_QuickStart.ipynb` and `Diblock.ipynb` first.

In [8]:
import os
os.environ["OMP_NUM_THREADS"] = "1"      # Single-threaded OpenMP
os.environ["MKL_NUM_THREADS"] = "1"      # Single-threaded MKL

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from polymerfts import PropagatorSolver
import polymerfts

## 2. Simulation Parameters

| Parameter | Value | Description |
|-----------|-------|-------------|
| $N$ | 100 | Reference chain length (number of statistical segments) |
| $\Delta s$ | 0.01 | Contour step size (= $1/N$) |
| $L_x, L_y$ | $5.0 R_0$ | Box dimensions in units of $R_0 = b\sqrt{N}$ |
| $m_x, m_y$ | 64 | Grid points per dimension |
| $b_A/b, b_B/b$ | 1.0 | Statistical segment lengths (relative to reference $b$) |

### Polymer Mixture Composition

| Polymer | Type | Volume Fraction | Length |
|---------|------|-----------------|--------|
| 0 | AB diblock | $\bar{\phi}_0 = 0.8$ | $\alpha_0 = 1.0$ ($f_A = 0.5$, $f_B = 0.5$) |
| 1 | A homopolymer | $\bar{\phi}_1 = 0.2$ | $\alpha_1 = 0.5$ |

In [9]:
# Simulation parameters
nx = [64, 64]              # Grid points
lx = [5.0, 5.0]            # Box size (in units of R0 = b*sqrt(N))
ds = 0.01                  # Contour step interval

# Block fractions for AB diblock
f_A = 0.5
f_B = 0.5

# Homopolymer length
f_homo = 0.5

## 3. Create the Solver

We use `PropagatorSolver` and add multiple polymers with their respective volume fractions.

**Polymer 0 (AB diblock):**
```
A(0)---A(1)---B(2)
```

**Polymer 1 (A homopolymer):**
```
A(0)---A(1)
```

In [10]:
# Create solver
solver = PropagatorSolver(
    nx=nx,
    lx=lx,
    ds=ds,
    bond_lengths={"A": 1.0, "B": 1.0},
    bc=["periodic", "periodic", "periodic", "periodic"],
    chain_model="continuous",
    method="pseudospectral",
    platform="cpu-mkl",
    reduce_memory=False
)

# Add AB diblock copolymer (80% volume fraction)
solver.add_polymer(
    volume_fraction=0.8,
    blocks=[
        ["A", f_A, 0, 1],
        ["B", f_B, 1, 2]
    ]
)

# Add A homopolymer (20% volume fraction)
solver.add_polymer(
    volume_fraction=0.2,
    blocks=[
        ["A", f_homo, 0, 1]
    ]
)

print(solver.info)

PropagatorSolver Configuration:
  Dimensions: 2D
  Grid: [64, 64]
  Box size: [5.0, 5.0]
  Boundary conditions: ['periodic', 'periodic', 'periodic', 'periodic']
  Chain model: continuous
  Contour step (ds): 0.01
  Method: pseudospectral
  Platform: cpu-mkl
  Monomer types: ['A', 'B']


## 4. Set Potential Fields

We create sinusoidal potential fields that are opposite for A and B monomers.

In [ ]:
# Create sinusoidal potential fields (opposite for A and B)
w_A = np.tile(np.sin(np.linspace(0, 2*np.pi, nx[0])), (nx[1], 1))
w_B = -w_A  # Opposite preference

# Visualize the potential fields
fig, axes = plt.subplots(1, 2, figsize=(10, 4))
fig.suptitle("Potential Fields", fontsize=14)

im0 = axes[0].imshow(w_A, extent=(0, lx[1], 0, lx[0]), origin='lower', cmap='viridis')
im1 = axes[1].imshow(w_B, extent=(0, lx[1], 0, lx[0]), origin='lower', cmap='viridis')
axes[0].set(title='$w_A$ (A monomers)', xlabel='y', ylabel='x')
axes[1].set(title='$w_B$ (B monomers)', xlabel='y', ylabel='x')

plt.colorbar(im0, ax=axes[0])
plt.colorbar(im1, ax=axes[1])
plt.tight_layout()
plt.show()

## 5. Compute Partition Functions

The propagators for A-type blocks, $(p,v,u) \in \{(0,0,1),(0,1,0),(1,0,1),(1,1,0)\}$:

$$\frac{\partial q^{v \rightarrow u}_p(\mathbf{r}, s)}{\partial s} = \frac{b_A^2}{6}\nabla^2 q^{v \rightarrow u}_p(\mathbf{r}, s) - w_A(\mathbf{r}) q^{v \rightarrow u}_p(\mathbf{r}, s), \quad s\in [0, f_A]$$

The propagators for B-type blocks, $(p,v,u) \in \{(0,1,2),(0,2,1)\}$:

$$\frac{\partial q^{v \rightarrow u}_p(\mathbf{r}, s)}{\partial s} = \frac{b_B^2}{6}\nabla^2 q^{v \rightarrow u}_p(\mathbf{r}, s) - w_B(\mathbf{r}) q^{v \rightarrow u}_p(\mathbf{r}, s), \quad s\in [0, f_B]$$

Single chain partition functions:

$$Q_0 = \frac{1}{V}\int d\mathbf{r}\, q^{0 \rightarrow 1}_0(\mathbf{r}, s) \cdot q^{1 \rightarrow 0}_0(\mathbf{r}, f_A-s)$$

$$Q_1 = \frac{1}{V}\int d\mathbf{r}\, q^{0 \rightarrow 1}_1(\mathbf{r}, s) \cdot q^{1 \rightarrow 0}_1(\mathbf{r}, f_A-s)$$

In [ ]:
# Compute propagators
solver.compute_propagators({"A": w_A.flatten(), "B": w_B.flatten()})

# Get single chain partition functions for each polymer
Q_0 = solver.get_partition_function(polymer=0)  # AB diblock
Q_1 = solver.get_partition_function(polymer=1)  # A homopolymer

print(f"Partition functions:")
print(f"  Q_0 (AB diblock) = {Q_0:.6f}")
print(f"  Q_1 (A homo)     = {Q_1:.6f}")

Partition functions:
  Q_0 (AB diblock) = 1.009877
  Q_1 (A homo)     = 1.059780

Note: Q_1 > Q_0 because the homopolymer has only A monomers,
      which can fully concentrate in favorable regions.


## 6. Compute Concentrations

$$\phi_A(\mathbf{r}) = \sum_{(p,v,u)\in\{(0,0,1),(1,0,1)\}} \frac{\bar{\phi}_p}{\alpha_p Q_p} \int_{0}^{f_A} ds\, q^{v \rightarrow u}_p(\mathbf{r}, s) \cdot q^{u \rightarrow v}_p(\mathbf{r}, f_A-s)$$

$$\phi_B(\mathbf{r}) = \sum_{(p,v,u)\in\{(0,1,2)\}} \frac{\bar{\phi}_p}{\alpha_p Q_p} \int_{0}^{f_B} ds\, q^{v \rightarrow u}_p(\mathbf{r}, s) \cdot q^{u \rightarrow v}_p(\mathbf{r}, f_B-s)$$

In [ ]:
# Compute concentrations
solver.compute_concentrations()

# Get total ensemble-averaged concentrations (sum over all polymers)
phi_A = np.reshape(solver.get_concentration("A"), nx)
phi_B = np.reshape(solver.get_concentration("B"), nx)

# Visualize total concentrations
vmin = min(phi_A.min(), phi_B.min())
vmax = max(phi_A.max(), phi_B.max())

fig, axes = plt.subplots(1, 2, figsize=(10, 4))
fig.suptitle("Total Concentrations (All Polymers)", fontsize=14)

im0 = axes[0].imshow(phi_A, extent=(0, lx[1], 0, lx[0]), origin='lower', 
                      cmap='RdBu_r', vmin=vmin, vmax=vmax)
axes[0].set(title='$\phi_A$ (total)', xlabel='y', ylabel='x')
plt.colorbar(im0, ax=axes[0])

im1 = axes[1].imshow(phi_B, extent=(0, lx[1], 0, lx[0]), origin='lower', 
                      cmap='RdBu_r', vmin=vmin, vmax=vmax)
axes[1].set(title='$\phi_B$ (total)', xlabel='y', ylabel='x')
plt.colorbar(im1, ax=axes[1])

plt.tight_layout()
plt.show()

In [ ]:
# Get concentrations for each polymer species separately
# (Using internal solver for polymer-specific access)

# AB diblock copolymer contributions
phi_A_diblock = np.reshape(solver._solver.get_total_concentration(0, "A"), nx)
phi_B_diblock = np.reshape(solver._solver.get_total_concentration(0, "B"), nx)

# A homopolymer contribution
phi_A_homo = np.reshape(solver._solver.get_total_concentration(1, "A"), nx)

# Visualize species-specific concentrations
fig, axes = plt.subplots(1, 3, figsize=(14, 4))
fig.suptitle("Species-Specific Concentrations", fontsize=14)

im0 = axes[0].imshow(phi_A_diblock, extent=(0, lx[1], 0, lx[0]), origin='lower', cmap='RdBu_r')
axes[0].set(title='$\phi_A$ (diblock)', xlabel='y', ylabel='x')
plt.colorbar(im0, ax=axes[0])

im1 = axes[1].imshow(phi_B_diblock, extent=(0, lx[1], 0, lx[0]), origin='lower', cmap='RdBu_r')
axes[1].set(title='$\phi_B$ (diblock)', xlabel='y', ylabel='x')
plt.colorbar(im1, ax=axes[1])

im2 = axes[2].imshow(phi_A_homo, extent=(0, lx[1], 0, lx[0]), origin='lower', cmap='RdBu_r')
axes[2].set(title='$\phi_A$ (homopolymer)', xlabel='y', ylabel='x')
plt.colorbar(im2, ax=axes[2])

plt.tight_layout()
plt.show()

print("\nObservation: The A homopolymer concentrates more strongly in favorable regions")
print("             because it's not constrained by connectivity to B blocks.")

## 7. Summary

In this tutorial, we learned:

1. **Polymer mixtures**: Multiple polymer species can be added with different volume fractions
2. **Partition functions**: Each species has its own partition function $Q_p$
3. **Concentration decomposition**: Total concentrations can be decomposed by species

### Next Steps

- **Branched polymers**: See `BranchedMultiArmStar.ipynb` and `BranchedComb.ipynb`
- **Non-periodic boundaries**: See `NonPeriodicBC.ipynb`
- **Accessing individual propagators**: Set `reduce_memory=False` to store all propagator steps (see `Diblock.ipynb` for examples)